# Optische Zeichenerkennung

![Ein Roboter liest eine Zeitung](./images/ocr.jpg)

Maschinelles Sehen wird oft eingesetzt, um Text in einem Bild zu erkennen und zu interpretieren. Diese Art der Verarbeitung wird oft auch als *optische Zeichenerkennung (Optical Character Recognition, OCR)* bezeichnet.

## Verwenden des Diensts für maschinelles Sehen, um Text in einem Bild zu lesen

Der Cognitive Service für **maschinelles Sehen** unterstützt verschiedene OCR-Aufgaben, darunter:

- Eine **OCR**-API, mit der Sie Text in mehreren Sprachen lesen können. Diese API kann synchron verwendet werden und eignet sich hervorragend, um kleine Mengen von Text in einem Bild zu erkennen und zu lesen.
- Eine **Lese**-API, die für größere Dokumente optimiert ist. Diese API wird asynchron verwendet und kann sowohl für gedruckten als auch für handgeschriebenen Text verwendet werden.

Sie können diesen Dienst verwenden, indem Sie entweder eine Ressource für **maschinelles Sehen** oder eine **Cognitive Services**-Ressource erstellen.

Falls noch nicht geschehen, erstellen Sie eine **Cognitive Services**-Ressource in Ihrem Azure-Abonnement.

> **Hinweis**: Falls Sie bereits eine Cognitive Services-Ressource haben, können Sie die entsprechende **Schnellstart**-Seite im Azure-Portal öffnen und den Schlüssel und den Endpunkt der Ressource unten in die Zelle kopieren. Führen Sie andernfalls die folgenden Schritte aus, um eine Ressource zu erstellen.

1. Öffnen Sie das Azure-Portal unter „https://portal.azure.com“ in einer neuen Browserregisterkarte, und melden Sie sich mit Ihrem Microsoft-Konto an.

2. Klicken Sie auf die Schaltfläche **&#65291;Ressource erstellen**, suchen Sie nach *Cognitive Services*, und erstellen Sie eine **Cognitive Services**-Ressource mit den folgenden Einstellungen:
    - **Abonnement**: *Ihr Azure-Abonnement*
    - **Ressourcengruppe**: *Wählen Sie eine Ressourcengruppe aus, oder erstellen Sie eine Ressourcengruppe mit einem eindeutigen Namen.*
    - **Region**: *Wählen Sie eine verfügbare Region aus*:
    - **Name**: *Geben Sie einen eindeutigen Namen ein.*
    - **Tarif**: S0
    - **Ich bestätige, dass ich die Hinweise gelesen und verstanden habe**: Ausgewählt
3. Warten Sie, bis die Bereitstellung abgeschlossen ist. Öffnen Sie anschließend Ihre Cognitive Services-Ressource, und klicken Sie auf der Seite **Übersicht** auf den Link zur Schlüsselverwaltung für den Dienst. Sie benötigen den Endpunkt und Schlüssel, um sich aus Clientanwendungen heraus mit Ihrer Cognitive Services-Ressource zu verbinden.

### Abrufen des Schlüssels und Endpunkts für Ihre Cognitive Services-Ressource

Um Ihre Cognitive Services-Ressource verwenden zu können, benötigen Clientanwendungen deren Endpunkt und Authentifizierungsschlüssel:

1. Kopieren Sie im Azure-Portal auf der Seite **Schlüssel und Endpunkt** für Ihre Cognitive Service-Ressource den **Schlüssel1** für Ihre Ressource, und fügen Sie ihn im unten stehenden Code anstelle von **YOUR_COG_KEY** ein.
2. Kopieren Sie den **Endpunkt** für Ihre Ressource, und fügen Sie ihn unten im Code anstelle von **YOUR_COG_ENDPOINT** ein.
3. Führen Sie die folgende Codezelle aus, indem Sie links neben der Zelle auf die Schaltfläche **Zelle ausführen** (&#9655;) klicken.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Nachdem Sie nun den Schlüssel und Endpunkt eingerichtet haben, können Sie Ihren Dienst für maschinelles Sehen verwenden, um Text aus Bildern zu extrahieren.

Wir beginnen zunächst mit der **OCR**-API, mit der Sie Bilder synchron analysieren und den enthaltenen Text lesen können. In diesem Fall verwenden wir ein Werbebild mit etwas Text für das fiktive Einzelhandelsunternehmen Northwind Traders. Führen Sie die folgende Zelle aus, um den Text zu lesen. 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

Der im Bild gefundene Text wird in eine hierarchische Struktur von Regionen, Zeilen und Wörtern eingeteilt. Der Code liest diese Struktur, um die Ergebnisse abzurufen.

In den Ergebnissen wird der gelesene Text über dem Bild angezeigt. 

## Anzeigen von Begrenzungsrahmen

Die Ergebnisse enthalten außerdem Koordinaten für *Begrenzungsrahmen* um die im Bild gefundenen Textzeilen und Wörter. Führen Sie die folgende Zelle aus, um die Begrenzungsrahmen für die Textzeilen aus dem oben verwendeten Werbebild anzuzeigen.

In [ ]:
# Öffnen Sie ein Bild, um es anzuzeigen.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

Im Ergebnis werden die Begrenzungsrahmen für die einzelnen Textzeilen als Rechteck im Bild angezeigt.

## Verwenden der Lese-API

Die zuvor verwendete OCR-API eignet sich gut für Bilder mit kleineren Mengen an Text. Für größere Textmengen wie etwa gescannte Dokumente können Sie die **Lese**-API verwenden. Hierzu sind mehrere Schritte erforderlich:

1. Übermitteln Sie ein Bild an den Dienst für maschinelles Sehen, um den Text asynchron zu lesen und zu analysieren.
2. Warten Sie, bis der Analysevorgang abgeschlossen ist.
3. Rufen Sie die Ergebnisse der Analyse ab.

Führen Sie die folgende Zelle aus, um mit diesem Vorgang den Text in einem gescannten Brief an den Filialleiter eines Northwind Traders-Geschäfts zu lesen.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

Überprüfen Sie die Ergebnisse. Sie erhalten eine komplette Abschrift des Briefs, der hauptsächlich aus gedrucktem Text und einer handschriftlichen Unterschrift besteht. Das ursprüngliche Bild des Briefs wird unter den OCR-Ergebnissen angezeigt. (Möglicherweise müssen Sie scrollen, um es zu sehen.)

## Lesen von handschriftlichem Text

Im vorherigen Beispiel haben wir in der Anforderung für die Bildanalyse einen Texterkennungsmodus angegeben, der den Vorgang für *gedruckten* Text optimiert. Beachten Sie, dass die handschriftliche Unterschrift trotzdem gelesen wurde.

Diese Fähigkeit, handschriftlichen Text zu lesen, ist extrem nützlich. Stellen Sie sich beispielsweise vor, Sie haben eine Einkaufsliste auf einen Notizzettel geschrieben und möchten Sie mit einer App auf Ihr Telefon einlesen und den enthaltenen Text erkennen.

Führen Sie die folgende Zelle aus, um einen Beispielvorgang für das Lesen einer handschriftlichen Einkaufsliste anzuzeigen.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Weitere Informationen

Weitere Informationen zur Verwendung des Cognitive Service für OCR finden Sie in der [Dokumentation zum maschinellen Sehen](https://docs.microsoft.com/de-de/azure/cognitive-services/computer-vision/concept-recognizing-text).